# Brands and Categories in Commentaries file

## Loading commentaries file and A/F file

In [1]:
# Imports
import numpy as np
import pandas as pd
from helper_save_load import load_excel_spreadsheet

In [ ]:
# Load Forecast and Actual sheets
dfc = load_excel_spreadsheet('./datasets/Commentaries dataset v2.xlsm', 'All')

In [ ]:
#Import dataframes to pickle file (saved previously)

from helper_save_load import load_from_pickle
df_a, df_f, df_v = load_from_pickle("dataframes_Dollars.pickle")
del df_a, df_v

In [ ]:
dfc.head(1)

In [ ]:
df_f['Category'][0:5]

In [ ]:
# df_f = df_v
df_f['Brand'][0:5]

In [ ]:
dfc['Category/ Brand'][0:5]

In [8]:
#Categories and brands in A/F file
categories_af = [category for category, values in df_f.groupby(['Category']).groups.items()]
brands_af = [brand for brand, values in df_f.groupby(['Brand']).groups.items()]

In [9]:
#Categories, brands and totals in Commentaries file
categories_brands = [category for category, values in dfc.groupby(['Category/ Brand']).groups.items()]
totals_c = [total for total in categories_brands if 'Total' in total]
categories_c = [category for category in categories_brands 
                if (category.upper() == category) | (('exc' in category) & (category not in totals_c))]
brands_c = [brand for brand in categories_brands if (brand not in categories_c) & (brand not in totals_c)]

## Applying fuzzy search to match categories and brands in both files

In [98]:
#Ucomment if python-Levenshtein not installed
#!pip install python-Levenshtein
#!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
#Example
print(fuzz.ratio('Deluxe Room, 1 King Bed', 'Deluxe King Room'))
print(fuzz.token_set_ratio('Deluxe Room, 1 King Bed', 'Deluxe King Room'))

62
100


In [100]:
def closest_label(target, labels_list, topn, threshold):
    import operator
    mydict = [(m, fuzz.token_set_ratio(target,m)) for m in labels_list]
    sorted_tuples = sorted(mydict, key=operator.itemgetter(1), reverse=True)
    result = [(item, score) for item, score in sorted_tuples if score>=threshold]
    return(result[0:topn])

def closet_label_list(items, labels_list, topn, threshold):
    for item in items:
        result = [m for m,s in closest_label(item, labels_list, topn, threshold)]
        output = item + '\t'
        output = output + '\t'.join(result)
        print(output)


target = categories_c[7]
labels_list = categories_af
print(target)
closest_label(target, labels_list, 3, 30)

HABL


[]

In [ ]:
closet_label_list(categories_c, categories_af, 3, 50)

In [ ]:
closet_label_list(brands_c, brands_af, 5, 50)

In [ ]:
totals_c2 = [word.replace('Total ', '') for word in totals_c]
closet_label_list(totals_c2, categories_af, 3, 50)

## Labelling commentaries by accurate brand names
Further to the feedback of Rob (Nov 20, 2018), commentaries are associated to more accurate brand labels from the forecast/actuals dataset

In [19]:
# Load association between brands in commentaries file and A/F files (by Rob)
dfb = load_excel_spreadsheet('./datasets/brands-categories-totals-updated.xlsx', 'Brands-processed')
dfb.drop(['Search result 1', 'Search result 2', 'Search result 3'], axis=1, inplace=True)

Sheets in file:  ['Brands-processed', 'Brands', 'Categories', 'Totals']
Data loaded from sheet: Brands-processed


In [ ]:
dfb.head(2)

In [25]:
dfb = dfb.rename(columns={'Brands in Commentaries file': 'Category/ Brand'})

In [ ]:
#Label each comment with the correct brand (brands that are not associated to A/F file are ignored)
dfbl = pd.merge(dfc, dfb, on='Category/ Brand')
dfbl.sort_values(['Num'], ascending=True, inplace=True)  #Sort values by original order
dfbl['Commentaries'].fillna('[NOC]', inplace=True)
dfbl['Commentaries vs Y-1'].fillna('[NOC]', inplace=True)
dfbl['Brand_2'].fillna('', inplace=True)
dfbl['Brand_3'].fillna('', inplace=True)
dfbl.drop(['Category', 'Brand'], axis=1, inplace=True)
dfbl.head(10)

In [84]:
#Save commentaries with brands dataframe to pickle file
from helper_save_load import save_to_pickle
save_to_pickle("commentaries_with_brands.pickle", (dfbl))

In [ ]:
#Comments containing the word driven
dfdr = dfbl[dfbl['Commentaries'].str.lower().str.contains("driven")]
dfdr